In [ ]:
from geemap import timelapse

In [ ]:
timeseries_defs = [x for x in dir(timelapse) if '_timeseries' in x and '_legacy' not in x and 'create_' not in x]
timeseries_defs

In [ ]:
from collections import defaultdict, Counter
import inspect

def get_defs_variables(module='timelapse', alike=['_timelapse'], notalike=['_legacy']):
    defs = [x for x in dir(eval(module)) 
                if all(y in x for y in alike) 
                and not any(y in x for y in notalike)]
    varnames = defaultdict(list)
    counter = Counter()
        
    for x in defs:
        for k in set(eval(f'inspect.getfullargspec({module}.{x})')[0]):
            varnames[k].append(x.split('_', 1)[0])
            counter[k] += 1
    return defs, counter, varnames

In [ ]:
par = {'module':'timelapse', 
       'alike':['naip_timeseries'], 
       'notalike':['_legacy', 'create_']}

defs, _, varnames = get_defs_variables(**par)
defs, len(defs), varnames

In [ ]:
par = {'module':'timelapse', 
       'alike':['_timeseries'], 
       'notalike':['_legacy', 'create_']}

defs, all_counter, varnames = get_defs_variables(**par)
defs, len(defs)

In [ ]:
par = {'module':'timelapse', 
       'alike':['create_timeseries'], 
       'notalike':[]}

_, create_counter, _ = get_defs_variables(**par)
create_counter

In [ ]:
{k:v for k,v in all_counter.items() if v == max(all_counter.values())}

In [ ]:
uncommon = {k:v for k,v in all_counter.items() if v != max(all_counter.values())}
uncommon

In [ ]:
outliers = {k:v for k,v in varnames.items() if len(v) != max(all_counter.values())}
outliers

In [ ]:
goes_vars = {
    "name": "{} {} {}",
    "def": "goes_timeseries",
    "pars": {
       "data":{"GOES-17":"GOES-17","GOES-16":"GOES-16"}, 
       "scan":{"Full disk":"full_disk", "Conus":"conus", "Mesoscale":"mesoscale"}, 
       "show_night":{"":[False, "a_mode"], "Night A":[True,"a_mode"], "Night B":[True, "b_mode"]},
       }
    }

goes_fire_vars = {
    "name": "{} Fire {}",
    "def": "goes_fire_timeseries",
    "pars": {
       "data":{"GOES-17":"GOES-17","GOES-16":"GOES-16"}, 
       "scan":{"Full disk":"full_disk", "Conus":"conus"}, 
       }
    }

naip_vars = {
    "name": "NAIP Annual {}",
    "def": "naip_ts",
    "pars": {
       "RGBN": {"":False, "4 Band":True},
       }
    }

landsat_vars = {
    "name": "Landsat {} {}",
    "def": "landsat_ts",
    "pars": {
       "frequency":{"Yearly":"year","Quarterly":"quarter","Monthly":"month"}, 
       "apply_fmask":{"Masked":True, "Raw":False,}, 
       }
    }

sentinel2_vars = {
    "name": "Sentinel2 {}{} {}",
    "def": "sentinel2_timeseries",
    "pars": {
       "frequency": {"Yearly":"year","Quarterly":"quarter","Monthly":"month"}, 
       "drop_empty": {"":False},
       "mask_cloud": {"Cloudless":True, "Clouded":False},
       }
    }

dynamic_vars = {
    "name": "Dynamic World {}{} {}",
    "def": "dynamic_world_timeseries",
    "pars": {
       "frequency": {"Yearly":"year","Quarterly":"quarter","Monthly":"month"}, 
       "drop_empty": {"":False}, 
       "return_type": {"Topological":"hillshade", "":"visualize", "Probabilistic":"probability"}
       }
    }

sentinel1_vars = {
    "name": "Sentinel1 {} {}",
    "def": "sentinel1_timeseries",
    "pars": {
       "frequency": {"Yearly":"year","Quarterly":"quarter","Monthly":"month"}, 
       "band": {"Land":"VV", "Sea and Ice":"HV"},
       }
    }

def parse_vars(dict_):
    from itertools import product
    for ki,di in dict_["pars"].items():
        dict_["pars"][ki] = list((ki,k,v) for k,v in di.items())
    products = product(*dict_["pars"].values())  
    full_ = []
    for sample in products:
        labels = [x[1] for x in sample]
        name = dict_["name"].format(*labels)
        params = {x[0]:x[2] for x in sample}
        full_.append((name, dict_["def"], params))
    return full_

vars_ = [goes_vars, goes_fire_vars, landsat_vars, sentinel1_vars, sentinel2_vars, naip_vars, dynamic_vars]

all_vars = [defx for var in vars_ for defx in parse_vars(var)]
all_vars

In [ ]:
from geemap import Map
Map = Map()
Map